In [154]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 150

In [156]:
def logits_function(p, X):
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [157]:
# Forward propagation
def forward_prop(params, X,y):
    logits = logits_function(params, X)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [158]:
def Get_PSO(f):
    # Initialize swarm
    options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

    # Call instance of PSO
    dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
    optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=1000)
    return pos

In [159]:
def predict(pos, X):
    logits = logits_function(pos, X)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [160]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

labelencoder = LabelEncoder()

def Start_Iris(b_ES = True):
    data = pd.read_csv('databases/iris.data', names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
    target = labelencoder.fit_transform(data['class'].values)
    data = data.drop('class', axis = 1).values

    def f(x):
        n_particles = x.shape[0]
        j = [forward_prop(x[i], data, target) for i in range(n_particles)]
        return np.array(j)

    pos = Get_PSO(f)
    scores = (predict(pos, data) == target)

    print("\n Iris \n")
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())

# def Start_Wine(b_ES = True):
#     data = pd.read_csv('databases/wine.data', names = ['class', 'alcohol', 'malic acid', 'ash', 'alcalinity of ash', 'magnesium', 'total phenols', 'flavanoids', 'nonflavanoid phenols', 'proanthocyanins', 'color intensity', 'hue', 'diluted', 'proline'])
#
#     target = data['class'].values
#     data_drop = data.drop('class',axis=1)
#     data = data_drop.values
#
#     sc = StandardScaler()
#     data = sc.fit_transform(data)
#
#     scores = []
#     vectors = []
#
#     centroids  = len(set(target))
#
#     if(b_ES):
#         for i in range(10):
#             best, score = es_plus(objective, bounds, n_iter, step_size, mu, lam, centroids, data, target)
#             scores.append(score)
#             vectors.append(best)
#     else:
#         for i in range(10):
#             solution = differential_evolution(pop_size, iter, F, cr, data, target, centroids)
#             scores.append(solution[1])
#             vectors.append(solution[0])
#
#     scores = np.array(scores)
#
#     print("\n Wine \n")
#     print("Mean: ", scores.mean())
#     print("Standard Deviation: ", scores.std())
#     print("Min: ", scores.min())
#     print("Max: ", scores.max())
#     print("Best vector", vectors[scores.argmax()])
#
# def Start_Breast_Cancer(b_ES = True):
#     data = pd.read_csv('databases/breast-cancer.data', names = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
#                                                                 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
#                                                                 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
#                                                                 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
#                                                                 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
#                                                                 'fractal_dimension_se', 'radius_worst', 'texture_worst',
#                                                                 'perimeter_worst', 'area_worst', 'smoothness_worst',
#                                                                 'compactness_worst', 'concavity_worst', 'concave points_worst',
#                                                                 'symmetry_worst', 'fractal_dimension_worst'])
#
#     data = data.drop('id',axis=1)
#
#     data['diagnosis'] = data['diagnosis'].map({'M':1,'B':0})
#
#     datas = pd.DataFrame(preprocessing.scale(data.iloc[:,1:31]))
#     datas.columns = list(data.iloc[:,1:31].columns)
#     target = data['diagnosis']
#     data = datas.values
#
#     scores = []
#     vectors = []
#
#     centroids = len(set(target))
#
#     if(b_ES):
#         for i in range(10):
#             best, score = es_plus(objective, bounds, n_iter, step_size, mu, lam, centroids, data, target)
#             scores.append(score)
#             vectors.append(best)
#     else:
#         for i in range(10):
#             solution = differential_evolution(pop_size, iter, F, cr, data, target, centroids)
#             scores.append(solution[1])
#             vectors.append(solution[0])
#
#     scores = np.array(scores)
#
#     print("\n Breast Cancer \n")
#     print("Mean: ", scores.mean())
#     print("Standard Deviation: ", scores.std())
#     print("Min: ", scores.min())
#     print("Max: ", scores.max())
#     print("Best vector", vectors[scores.argmax()])

In [161]:
Start_Iris()

2022-03-06 15:08:37,111 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.0182
2022-03-06 15:09:07,531 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.018221833148987576, best pos: [ 5.64704657e-01  1.77363594e-01 -1.62001346e+00  9.64797210e-01
  2.88935153e-01  5.20451980e-01  5.94741293e-01 -3.35928187e-01
 -1.01324648e+00 -1.87210349e+00 -1.82650788e+00  1.54800755e+00
 -3.44260446e-01  1.59102564e+00  1.00601672e+00 -9.62158742e-01
  7.21091214e-01  5.19251159e-01  2.56329257e+00  4.17093168e-01
 -9.85735097e-01 -2.58215376e+00 -1.07653877e+00  8.36166083e-01
 -1.70580313e+00  3.86526885e-01 -8.07632344e-01  4.17660690e+00
 -8.21354256e-01 -3.16169759e-01 -2.24849623e+00 -3.01756903e-01
  1.66037796e+00  2.46396550e+00 -7.88877444e-01 -9.68119439e-01
 -3.74135011e-01  5.33961618e-01 -5.55836088e-01  6.37544254e+00
  9.68201134e-01  7.61


 Iris 

Mean:  0.9866666666666667
Standard Deviation:  0.11469767022723505
Min:  False
Max:  True
